# Moteur de recommandations de films

In [21]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


 
def get_recommendations(title, cosine_sim, indices):
    val=None
    #Obtenir l'index du film qui correspond au titre à l'aide de la clé de titre des index.
    index = indices[title]
    
    # trier les films à partir des "similarity scores"
    sim_scores = list(enumerate(cosine_sim[index]))
    sim_scores = sorted(sim_scores,key = lambda x: x[1], reverse=True)
    
    # obtenir les scores des 5 films les plus proches
    if sim_scores[0][0]!=index:
        for i in sim_scores:
            if i[0]==index:
                print("ok")
                sim_scores.remove(i)
                sim_scores.insert(0,i)
    sim_scores = sim_scores[1:6]

    
    # obtenir les indices
    
    movie_indices = [i[0] for i in sim_scores]
    
    # renvoyer le top 10 des films similaires
    
    return movies['movie_title'].iloc[movie_indices]



In [22]:
#moteur de reccommandations 
#Lecture du fichier et nettoyage des données


movies = pd.read_csv('movie_metadata.csv')
movies['movie_title'] = movies['movie_title'].str.rstrip()

movies = movies.drop_duplicates(subset = ['movie_title','title_year'],keep='first')


#vectorisation
indices = pd.Series([i for i in range(len(movies))], index=movies['movie_title']).drop_duplicates()
vectorizer = TfidfVectorizer()

#Titre
tfidf_matrix_title = vectorizer.fit_transform(movies['movie_title'])
cosine_sim_title = linear_kernel(tfidf_matrix_title, tfidf_matrix_title)

#genre
tfidf_matrix_genre = vectorizer.fit_transform(movies['genres'])
cosine_sim_genre = linear_kernel(tfidf_matrix_genre, tfidf_matrix_genre)

#Premier acteur
tfidf_matrix_actor1 = vectorizer.fit_transform(movies["actor_1_name"].values.astype('U'))
cosine_sim_actor1 = linear_kernel(tfidf_matrix_actor1, tfidf_matrix_actor1)


In [4]:
print(get_recommendations("Avatar",cosine_sim_genre,indices))
print()
print(get_recommendations("Paddington",cosine_sim_title,indices))
print()
print(get_recommendations("Garfield",cosine_sim_title+cosine_sim_genre,indices))

119
402      Hotel Transylvania
478    Hotel Transylvania 2
789              Garfield 2
870              Paddington
954                Garfield
Name: movie_title, dtype: object
119
0                                        Avatar
1      Pirates of the Caribbean: At World's End
2                                       Spectre
3                         The Dark Knight Rises
4    Star Wars: Episode VII - The Force Awakens
Name: movie_title, dtype: object
119
402      Hotel Transylvania
478    Hotel Transylvania 2
789              Garfield 2
870              Paddington
954                Garfield
Name: movie_title, dtype: object


# Les tests

In [23]:
#test avec 25 films différents
from random import *

#Selection des 25 films différents
liste_titre_films = []
while len(liste_titre_films) < 25:
    choix_film = choice(list(movies['movie_title']))
    while choix_film in liste_titre_films :
        choix_film = choice(list(movies['movie_title']))
    liste_titre_films.append(choix_film)
    
choix = int(input("Choisissez quelle type de recommandation voulez vous tester (1:titre / 2:genre / 3:acteur1 ):"))

#affichage des recommandation des 25 films
for titre in liste_titre_films :
    print("Recommandation film",titre,':')
    if choix==1:
        print(get_recommendations(titre,cosine_sim_title,indices))
    elif choix == 2:
        print(get_recommendations(titre,cosine_sim_genre,indices))
    elif choix == 3:
        print(get_recommendations(titre,cosine_sim_actor1,indices))
    print()

Choisissez quelle type de recommandation voulez vous tester (1:titre / 2:genre / 3:acteur1 ):1
Recommandation film Iraq for Sale: The War Profiteers :
1951                           War
3354    Valley of the Wolves: Iraq
46                     World War Z
3645                    Men of War
3013                 5 Days of War
Name: movie_title, dtype: object

Recommandation film Bicentennial Man :
2296                    The Man
548     The Man from U.N.C.L.E.
946                  Inside Man
1012           Man of the House
3285               The Best Man
Name: movie_title, dtype: object

Recommandation film A Warrior's Tail :
1978               Warrior
215       The 13th Warrior
1174     The Warrior's Way
3196      Peaceful Warrior
1324    Nomad: The Warrior
Name: movie_title, dtype: object

Recommandation film Cheaper by the Dozen 2 :
1193    Cheaper by the Dozen
2333              By the Sea
1285         City by the Sea
3806               Flying By
3362             Stand by Me
Name: mov